In [1]:
!pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface

In [2]:
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api

In [3]:
%pip install llama-index-readers-file

In [4]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.node_parser import SimpleFileNodeParser
import warnings
import os
# from neo4j import GraphDatabase
import spacy


In [5]:
from huggingface_hub import notebook_login
notebook_login()


In [6]:
data1 = '/content/majmaolbayan.txt'

data2 = '/content/alborhan.txt'

In [7]:
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(
    chunk_size=512,
    chunk_overlap=100,
)


In [8]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex

documents = SimpleDirectoryReader(input_files=[data1, data2]).load_data()

In [9]:
%pip install llama-index-embeddings-jinaai
%pip install llama-index-llms-openai

In [10]:
from llama_index.embeddings.jinaai import JinaEmbedding

embed_model1 = JinaEmbedding(
    model="jina-embeddings-v3",
    # embed_batch_size=16,
    # task="retrieval.passage",
)



In [11]:

embed_model = HuggingFaceEmbedding(model_name='PartAI/Tooka-SBERT-V2-Large')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
index = VectorStoreIndex.from_documents(documents,embed_model=embed_model,transformations=[node_parser])

KeyboardInterrupt: 

In [39]:
# from llama_index import StorageContext
index.storage_context.persist(persist_dir="/content/my_index_storage")


In [43]:
from google.colab import files
files.download("/content/my_index_storage")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
def answer_query(query, index, openrouter_api_key, top_k=5):
    retriever = index.as_retriever()
    nodes = retriever.retrieve(query)
    context = "\n\n".join([n.text for n in nodes[:top_k]])

    prompt = f"""
تو یک دستیار فارسی زبان هستی که به سوالات بر اساس استناد به قرآن پاسخ جامع و کامل میدهی

    اطلاعات:
    {context}

    سوال:
    {query}

اگر سوال کاربر نامرتبط به دین و قران بود بگو که نمیتوانی پاسخ دهی.همچنین مرحله فکر کردنت را به کاربر نشان نده
    """

    from openai import OpenAI
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key='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',
    )
    completion = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content


In [65]:
response = answer_query('آیا نام امام علی در قرآن آمده است؟', index, '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')
print(response)



با استناد به اطلاعات ارائه شده، نام امام علی (ع) به صورت مستقیم در قرآن نیامده است، اما در قرآن نام‌هایی برای آن حضرت وجود دارد که مردم از آن آگاه نیستند. 

به عنوان مثال، در آیه «وَمَن یَکْفُرْ بِالإِیمَانِ فَقَدْ حَبِطَ عَمَلُهُ» (سوره مائده)، واژه "ایمان" به امام علی (ع) تأویل شده است، به این معنا که ایمان به ولایت آن حضرت مورد نظر است. همچنین در آیه «أَفَمَن کَانَ عَلَی بَیِّنَةٍ مِّن رَّبِّهِ وَیَتْلُوهُ شَاهِدٌ مِّنْهُ» (سوره فصلت)، "شاهد" به امیرمؤمنان علی (ع) تفسیر شده است. علاوه بر این، اشاره به ولایت آن حضرت در کتاب موسی (ع) نیز در قرآن وجود دارد.



In [54]:
response = answer_query('ناهار امروز چی خوردی؟', index, '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')
print(response)


متاسفم، من نمی‌توانم به این سوال پاسخ دهم. سوال شما مربوط به زندگی شخصی من است و من یک دستیار مجازی هستم که برای پاسخ به سوالات بر اساس قرآن و متون دینی طراحی شده‌ام.


In [56]:
response = answer_query('حقوق زنان در قرآن چیست؟', index, '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')
print(response)



حقوق زنان در قرآن، به طور جامع و کامل در آیه «وَلَهُنَّ مِثْلُ الَّذِی عَلَیْهِنَّ بِالْمَعْرُوفِ» بیان شده است. این آیه می‌فرماید: «برای زنان، حقوقی است برابر با حقوقی که مردان بر آن‌ها دارند، به شیوه‌ای پسندیده و عادلانه».

بنابراین، حقوق زنان در مقابل مردان، مشابه حقوق مردان در مقابل زنان است و شامل موارد زیر می‌شود:

*   **نیکی و معاشرت حسنه:** شوهر باید با همسر خود به خوبی رفتار کند و با او خوش‌رفتاری نماید.
*   **عدم آزار و اذیت:** شوهر نباید به همسر خود آسیب جسمی یا روحی برساند.
*   **تساوی در بهره‌مندی از زندگی مشترک:** زن و مرد باید در مسائل مربوط به زندگی مشترک، به طور عادلانه با یکدیگر رفتار کنند.
*   **نفقه و پوشاک:** شوهر موظف به تامین نفقه و پوشاک همسر خود است.

همچنین، قرآن در آیاتی دیگر به حقوق دیگری از زنان اشاره می‌کند، از جمله:

*   **حق در کسب و تصرف اموال:** زن می‌تواند مالک اموال خود باشد و در آن‌ها تصرف کند.
*   **حق در طلاق:** زن در شرایط خاصی حق درخواست طلاق را دارد.
*   **حق در ارث:** زن حق دارد از ترکه پدر و همسر خود ارث ببرد.
*   **حق در آموزش و تحصیل:** ز

In [57]:
response = answer_query('با توجه به آیات قرآن جانشین پیامبر بعد از وی کیست؟', index, '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')
print(response)

با استناد به متن ارائه شده، جانشین پیامبر صلی الله علیه و آله و سلم، حجت پس از ایشان است. این شخص در اختلافات امت راهنماست، حقوق مردم را می‌ستاند، اوامر الهی را اجرا و عدالت را برقرار می‌کند. 

متن به آیه 53 سوره فصلت استناد می‌کند و می‌گوید اگر کسی احکام پروردگار را اجرا نکند، نشانه دیگری در جهان وجود ندارد که خداوند آن را نشان دهد. سپس با طرح این سوال که کدام نشانه الهی از حجت بزرگتر است، بر جایگاه و اهمیت جانشین پیامبر تاکید می‌کند.

همچنین، متن به این نکته اشاره دارد که رسول خدا صلی الله علیه و آله و سلم تنها کسی را جانشین خود می‌کند که به حکم ایشان حکم کند و در همه چیز جز پیامبری همانند ایشان باشد. اگر چنین جانشینی تعیین نشده باشد، کسانی که پس از پیامبر می‌آیند، تباه خواهند شد.



In [58]:
response = answer_query('حجاب در قرآن', index, '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')
print(response)



با استناد به اطلاعات ارائه شده، حجاب در قرآن معانی متعددی دارد و تفسیرهای مختلفی را می‌پذیرد. این مفهوم هم به معنای پوشاندن فیزیکی و هم به معنای پرده‌ای معنوی و غیبی به کار رفته است.

*   **حجاب به معنای پوشش:** در لغت، حجاب به معنای پوشاندن است و در روایات، حجاب به عنوان پرده و پوششی برای خداوند و واسطه‌ای بین او و خلق ذکر شده است. به عنوان مثال، در روایت ابوذر از پیامبر اکرم (ص)، علی (ع) به عنوان حجاب بین خدا و خلق معرفی شده است.
*   **حجاب به عنوان مانع:** در آیه 9 سوره یس، حجاب به عنوان مانعی بین پیامبر (ص) و کسانی که به آخرت ایمان ندارند توصیف شده است. این آیه به پنج حجاب اشاره می‌کند که نشان‌دهنده درجات مختلفی از مانع و پرده‌داری است.
*   **حجاب در ابتدای سوره‌ها:** روایتی از عیاشی از زید بن علی نقل شده که به تفسیر آیه «بسم الله الرحمن الرحیم» پرداخته و مفهوم حجاب را در ارتباط با آن بررسی می‌کند (جزئیات بیشتر در روایت ذکر نشده است).

به طور کلی، می‌توان گفت حجاب در قرآن یک مفهوم چندوجهی است که بسته به سیاق، می‌تواند به پوشش فیزیکی، پرده غیبی، مانع معنوی یا واسطه بین خدا و خلق ا

In [59]:
response = answer_query('معنای «یَمْحُو اللّهُ مَا یَشَاءُ وَیُثْبِتُ وَعِندَهُ أُمُّ الْکِتَابِ» چیست؟', index, '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')
print(response)

معنای آیه «یَمْحُو اللّهُ مَا یَشَاءُ وَیُثْبِتُ وَعِندَهُ أُمُّ الْکِتَابِ» (رعد: 39) این است که خداوند اراده و قدرت دارد هر آنچه را که بخواهد محو کند و هر آنچه را که بخواهد تثبیت نماید. این محو و تثبیت، هم در مورد علم و تقدیرات الهی و هم در مورد احکام و تکالیف شریعت قابل تصور است.

همانطور که در روایات نیز اشاره شده، خداوند می‌تواند آنچه را که هنوز واقع نشده، به وقوع برساند و آنچه را که واقع شده، از بین ببرد یا تغییر دهد. این امر نشان‌دهنده علم و قدرت بی‌نهایت خداوند است.

همچنین، «عِندَهُ أُمُّ الْکِتَابِ» به این معناست که اصل و سرچشمه تمام وقایع و تقدیرات، در نزد خداوند است و همه چیز در علم ازلی او ثبت و محفوظ است. این کتاب، همان «کتاب محفوظ» است که در آیه دیگری به آن اشاره شده و خداوند بر آن علم دارد و ثبت آن بر او آسان است.

به عبارت دیگر، خداوند هم علم به تمام آنچه بوده و خواهد بود دارد و هم اراده و قدرت تغییر و تحول در آن را داراست. این آیه، تأکیدی بر یگانگی، علم و قدرت بی‌کران خداوند است.



In [60]:
response = answer_query('رمز موفقیت در زندگی', index, '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')
print(response)

با استناد به آیات و روایات ارائه شده، رمز موفقیت در زندگی را می‌توان در چند نکته کلیدی خلاصه کرد:

**1. تعادل در زندگی:**

همانطور که در روایت اول به آن اشاره شده، خردمند کسی است که ساعات عمر خود را به درستی تقسیم کند. این تقسیم‌بندی شامل موارد زیر است:

*   **عبادت و راز و نیاز با خدا:** اختصاص دادن زمانی برای ارتباط با پروردگار، آرامش و جهت‌دهی به زندگی می‌بخشد.
*   **خودسازی و محاسبه نفس:** بررسی اعمال و رفتار خود و تلاش برای اصلاح آن‌ها، فرد را به کمال می‌رساند.
*   **شکرگزاری:** تفکر در نعمت‌های خداوند، باعث افزایش رضایت و خوشبختی در زندگی می‌شود.
*   **بهره‌مندی حلال:** استفاده از لذت‌های مشروع و حلال، به تقویت روحیه و نشاط در زندگی کمک می‌کند.

**2. عمل صالح و پیروی از خواسته‌های خدا:**

در روایت دوم، امام صادق (ع) می‌فرمایند: «هر گاه بنده ای در راه کاری که خداوند آن را دوست دارد گام بردارد، خداوند نیز کاری را که شخص دوست می دارد انجام خواهد داد.» این بدان معناست که با انجام اعمال صالح و پیروی از دستورات الهی، زمینه برای تحقق اهداف و آرزوهای خود را فراهم می‌کنیم.

**3. توکل و اع

In [61]:
response = answer_query('چرا در شهادت، دو زن به جاى یک مرد حساب مى شود؟', index, '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')
print(response)



با استناد به آیات قرآن و روایات، پاسخ به این سوال به شرح زیر است:

دلیل برابری شهادت دو زن با یک مرد، در واقع به ضعف حافظه زنان باز می‌گردد. در آیه مربوط به شهادت در معاملات و بدهی‌ها، خداوند می‌فرماید: «یان تقضوا الدّین أو تأخذوا حظّاً مما خلق الله لكم في الأرض و ما منكم من آمن إلاّ أن يتقّیٰ» (یونس، آیه 94) و در توضیح این آیه، مفسران اشاره می‌کنند که هدف از ذکر دو زن به جای یک مرد به عنوان شاهد، این است که اگر یکی از آن‌ها فراموش کرد، دیگری می‌تواند او را یادآوری کند.

همچنین، در آیه دیگر، خداوند می‌فرماید: «و اشهدوا ذوی عدل منکم» (نمل، آیه 4) که در آن بر حضور دو شاهد عادل تاکید شده است. با توجه به اینکه زنان به طور طبیعی بیشتر از مردان فراموشکار هستند، برای اطمینان از صحت گواهی و جلوگیری از اشتباه، نیاز به دو شاهد زن به جای یک شاهد مرد است. 

این موضوع به معنای بی‌ارزش بودن شهادت زنان نیست، بلکه یک واقعیت طبیعی را در نظر می‌گیرد و برای جبران آن، راهکاری ارائه می‌دهد.
در مورد خنثی‌ها نیز، امام علی (ع) فرموده‌اند که خنثی از شرمگاه خود ارث می‌برد و شهادت او به نفع خودش پذیرفته نمی‌شو

In [62]:
response = answer_query('درباره ارث و میراث در قرآن توضیح بده؟', index, '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')
print(response)



میراث و ارث در قرآن کریم، موضوعی مهم و دارای احکام جزئی و دقیق است. قرآن در آیه 11 سوره نساء به طور مستقیم به مسئله تقسیم ارث می‌پردازد و احکامی را در این زمینه بیان می‌کند. این آیه در مورد تعیین سهم‌المیراث برای فرزندان، همسر و پدر و مادر است.

**نکات کلیدی از آیه ارث:**

*   **توزیع عادلانه:** هدف اصلی از احکام ارث در اسلام، ایجاد عدالت و حفظ حقوق همه وارثین است.
*   **تفاوت سهم‌ها:** سهم ارث پسر دو برابر سهم دختر است (لِلذَّکَرِ مِثْلُ حَظِّ الْأُنْثَیَیْنِ).
*   **تعداد وارثین:** احکام مربوط به ارث با توجه به تعداد و نوع وارثین (پسر، دختر، همسر، پدر، مادر و غیره) متفاوت است.
*   **دختران بیشتر از دو نفر:** اگر تعداد دختران بیشتر از دو نفر باشد، سهم آن‌ها به نسبت تقسیم می‌شود (فَإِنْ کُنَّ نِسٰاءً فَوْقَ اثْنَتَیْنِ).

**تأویل وارث و میراث:**

در متون دینی، اشاره‌ای به تأویل واژه "وارث" و "میراث" وجود دارد. بر اساس این تأویل، می‌توان واژه "تراث" را در آیات مربوط به ارث، به میراث پیامبر (صلی الله علیه وآله) و ائمه اطهار (علیهم السلام) تعبیر کرد. به این معنا که منافع خلافت و مواردی 

In [64]:
response = answer_query('مگر خداوند در قران نفرموده که خدا برای بنده کافی است پس چرا شیعیان به امامان توسل میکنند؟', index, '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')
print(response)

با استناد به آیات قرآن و روایتی که ارائه دادید، پاسخ این سوال به شرح زیر است:

خداوند در قرآن کریم به صراحت فرموده است که او برای بندگانش کافی است و نیازی به غیر او نیست. آیه «و من یتوکل علی الله فکفاه» (طلاق: 3) به همین موضوع اشاره دارد. همچنین در روایت ذکر شده، وقتی از امام جعفر صادق (ع) پرسیده شد که چرا در سوره حمد از خداوند استعانت می‌طلبیم، ایشان فرمودند: اگر امور به تو سپرده شده، چه نیازی به یاری دیگران داری؟

اما سوال شما درباره توسل شیعیان به امامان (ع) است. در پاسخ باید گفت که توسل به امامان (ع) به معنای پرستش یا شرک به خدا نیست، بلکه به معنای درخواست واسطه از کسانی است که نزد خداوند منزلت بالایی دارند. شیعیان معتقدند که امامان (ع) واسطه فیض الهی هستند و می‌توانند دعای بندگان را به درگاه خداوند ببرند.

این اعتقاد با آیه قرآن «یا أیها الذین آمنوا اتقوا الله و ابتغوا الیه وسیلة» (مائده: 35) نیز سازگار است. در این آیه، خداوند مؤمنان را به تقوا و جستجوی وسیله به سوی خود دعوت می‌کند. شیعیان، امامان (ع) را همان وسیله می‌دانند که خداوند به مؤمنان معرفی کرده است.

همچنین، در روایت آمد